### MCOE Compilation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pudl
import pandas as pd
import sqlalchemy as sa
import sys
import logging
import pathlib
import json

In [3]:
# this basic setup for logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pd.options.display.max_columns = None

In [4]:
# pudl_settings is a dictionary that includes the paths to several key pudl directories
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [5]:
# the creation of the pudl_out object
# this will compile output tables like mcoe - if you want to restart/wipe the saved elements rerun this cell
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, freq='AS', rolling=True)

In [10]:
# run mcoe method... the original function is in pudl.analysis.mcoe.mcoe()
mcoe = pudl_out.mcoe()

In [11]:
mcoe_plant_level = mcoe[['plant_id_eia',
                         'plant_id_pudl',
                         'plant_name_eia',
                         'latitude',
                         'longitude',
                         'state',
                         'city']]

mcoe_unit_level = mcoe[['plant_id_eia',
                        'plant_id_pudl',
                        'generator_id',
                        'nameplate_power_factor',
                        'generator_age_years',
                        'retirement_date',
                        'planned_retirement_date',
                        'technology_description',
                        'energy_source_code_1',
                        'fuel_type_code_pudl',
                        'total_mmbtu',
                        'net_generation_mwh',
                        'heat_rate_mmbtu_mwh']]

In [12]:
mcoe_plant_level

,plant_id_eia,plant_id_pudl,plant_name_eia,latitude,longitude,state,city
0,3,32,Barry,31.0069,-88.0103,AL,Bucks
1,3,32,Barry,31.0069,-88.0103,AL,Bucks
2,3,32,Barry,31.0069,-88.0103,AL,Bucks
3,3,32,Barry,31.0069,-88.0103,AL,Bucks
4,3,32,Barry,31.0069,-88.0103,AL,Bucks
...,...,...,...,...,...,...,...
8855928,59784,236,Gila River Power Block 3,32.9750,-112.6944,AZ,Gila Bend
8855929,59784,236,Gila River Power Block 3,32.9750,-112.6944,AZ,Gila Bend
8855930,59784,236,Gila River Power Block 3,32.9750,-112.6944,AZ,Gila Bend
8855931,59784,236,Gila River Power Block 3,32.9750,-112.6944,AZ,Gila Bend
